In [1]:
import os
from langchain import LLMMathChain, SerpAPIWrapper
from langchain.agents import Tool, initialize_agent
from langchain.llms import OpenAI
import langchain
langchain.verbose = True
# LangChain Tracing
from langchain.callbacks import get_callback_manager, set_tracing_callback_manager
from langchain.callbacks.tracers import SharedLangChainTracer
set_tracing_callback_manager()

In [2]:
llm = OpenAI(temperature=0)
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain(llm=llm, verbose=True)
tools = [
    Tool(
        name="Search",
        func=search.run,
        description="useful for when you need to answer questions about current events",
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
    ),
]

agent = initialize_agent(
    tools, llm, agent="zero-shot-react-description", verbose=True
)

In [14]:
tracer_session = SharedLangChainTracer().new_session()
print(f"New tracing session: {tracer_session}")

agent.run(
    "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"
)

POST http://127.0.0.1:5000/sessions, status code: 201, id: 1
New tracing session: TracerSession(id=1, start_time=datetime.datetime(2023, 1, 3, 23, 15, 27, 843604), extra={})


> Entering new AgentExecutor chain...


> Entering new LLMChain chain...
Prompts after formatting:
Answer the following questions as best you can. You have access to the following tools:

Search: useful for when you need to answer questions about current events
Calculator: useful for when you need to answer questions about math

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Search, Calculator]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

Question: Who won the US Open men's tennis final in

"Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."

In [15]:
tracer_session.to_dict()

{'id': 1,
 'start_time': datetime.datetime(2023, 1, 3, 23, 15, 27, 843604),
 'extra': {}}

In [16]:
import requests

r = requests.get("http://127.0.0.1:5000/sessions")
r.json()

[{'id': 1, 'start_time': '2023-01-03T23:15:27.843604', 'extra': {}}]

In [17]:
SharedLangChainTracer().load_session(1)

TracerSession(id=1, start_time=datetime.datetime(2023, 1, 3, 23, 15, 27, 843604, tzinfo=datetime.timezone(datetime.timedelta(days=-1, seconds=57600), 'PST')), extra={})

In [18]:
SharedLangChainTracer().load_session(2) # Expect exception -- this session does not exist

TracerException: Failed to load session 2

In [28]:
r = requests.get("http://127.0.0.1:5000/chain-runs?id=1")
r.json()

{'id': 1,
 'start_time': 1672816527.971262,
 'end_time': 1672816540.058254,
 'extra': {},
 'execution_order': 1,
 'serialized': {'name': 'AgentExecutor'},
 'session_id': 1,
 'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"},
 'outputs': {'output': "Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."},
 'child_runs': [{'id': 4,
   'start_time': 1672816527.977814,
   'end_time': 1672816534.373032,
   'extra': {},
   'execution_order': 2,
   'serialized': {'name': 'LLMChain'},
   'session_id': 1,
   'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
    'agent_scratchpad': '',
    'stop': ['\nObservation: ']},
   'outputs': {'text': ' I need to find out who won the US Open men\'s tennis final in 2019 and then calculate his age raised to the second power.\nAction: Search\nAction Input: "US Open men\'s

In [30]:
r = requests.get("http://127.0.0.1:5000/llm-runs?top_level=True&session_id=1")
r.json()

[]

In [31]:
r = requests.get("http://127.0.0.1:5000/tool-runs?session_id=2")
r.json()

[]

In [34]:
r = requests.get("http://127.0.0.1:5000/chain-runs")
print(f"{len(r.json())} total chain runs")
r.json()

7 total chain runs


[{'id': 1,
  'start_time': '2023-01-03T23:15:27.971262',
  'end_time': '2023-01-03T23:15:40.058254',
  'extra': {},
  'execution_order': 1,
  'serialized': {'name': 'AgentExecutor'},
  'session_id': 1,
  'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"},
  'outputs': {'output': "Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."},
  'child_runs': [{'id': 4,
    'start_time': '2023-01-03T23:15:27.977814',
    'end_time': '2023-01-03T23:15:34.373032',
    'extra': {},
    'execution_order': 2,
    'serialized': {'name': 'LLMChain'},
    'session_id': 1,
    'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
     'agent_scratchpad': '',
     'stop': ['\nObservation: ']},
    'outputs': {'text': ' I need to find out who won the US Open men\'s tennis final in 2019 and then calculate his age raised to 

In [35]:
r = requests.get("http://127.0.0.1:5000/chain-runs?top_level=True")
print(f"{len(r.json())} total top level runs")
r.json()

1 total top level runs


[{'id': 1,
  'start_time': '2023-01-03T23:15:27.971262',
  'end_time': '2023-01-03T23:15:40.058254',
  'extra': {},
  'execution_order': 1,
  'serialized': {'name': 'AgentExecutor'},
  'session_id': 1,
  'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?"},
  'outputs': {'output': "Rafael Nadal, aged 36, won the US Open men's tennis final in 2019 and his age raised to the second power is 1296."},
  'child_runs': [{'id': 4,
    'start_time': '2023-01-03T23:15:27.977814',
    'end_time': '2023-01-03T23:15:34.373032',
    'extra': {},
    'execution_order': 2,
    'serialized': {'name': 'LLMChain'},
    'session_id': 1,
    'inputs': {'input': "Who won the US Open men's tennis final in 2019? What is his age raised to the second power?",
     'agent_scratchpad': '',
     'stop': ['\nObservation: ']},
    'outputs': {'text': ' I need to find out who won the US Open men\'s tennis final in 2019 and then calculate his age raised to 